<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-27 14:26:54
-------------------
qualified stocks: 93
with latest results: 29
still star stocks: 18


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: 40.47 K (0.3%)
Current PnL: -15.17 L (-10.57%)
CY Booked + Current PnL: -8.43 L (-5.87%)
-------------------
Total profit:  3.57 L
Total loss:  -18.74 L
-------------------
Total Booked + Current PnL: 18.55 L (15.92%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.97%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 73.09 L (53.88%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.42%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,53.76,102659.0,24297.0,9732.0,1.16,31.01,9.48,43.43,77.0,2.50,0.76,37.97,XY24,NTT,MISC
36,HINDALCO,651.95,761.55,-19.08,H-LC,74.19,111648.0,7336.0,10205.0,1.04,7.03,9.14,16.81,27.0,0.72,0.82,24.21,X5K,ATH,METALS
85,UJJIVANSFB,52.77,53.00,52.86,M-SC,84.95,132138.0,-10341.0,10967.0,-0.33,-7.26,8.30,0.44,243.0,-0.94,0.97,55.96,OX40N,NTT,BANKS
79,TATAELXSI,7332.28,7332.00,-15.63,X-MC,33.33,88627.0,-14025.0,14021.0,0.89,-13.66,15.82,-0.00,58.0,-1.00,0.65,34.72,OX40N,NTT,IT
90,VIPIND,488.80,489.00,-212.52,H-SC,96.77,81140.0,-14176.0,14216.0,-0.14,-14.87,17.52,0.04,154.0,-1.00,0.60,60.04,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,60.22,180638.0,830.0,111580.0,0.43,0.46,61.77,62.52,171.0,0.01,1.33,34.71,XR,NTT,BANKS
52,KPIGREEN,497.21,732.16,20.81,H-SC,70.43,124992.0,-305.0,59509.0,-0.23,-0.24,47.61,47.25,139.0,-0.01,0.92,56.37,MH,ATH,POWER
51,KPIGREEN,497.21,732.16,22.30,H-SC,70.43,124992.0,-305.0,59509.0,-0.23,-0.24,47.61,47.25,139.0,-0.01,0.92,56.37,MH,ATH,POWER
15,BLUESTARCO,1646.70,2337.55,-12.48,H-SC,70.43,164090.0,-580.0,69673.0,-1.20,-0.35,42.46,41.95,121.0,-0.01,1.21,7.09,X40N,ATH,AC
35,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,34.41,151624.0,711.0,59740.0,1.26,0.47,39.40,40.06,102.0,0.01,1.12,24.02,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,CERA,8421.60,8422.00,-14.08,X-SC,68.82,88725.0,-20756.0,20762.0,3.46,-18.96,23.40,0.00,68.0,-1.00,0.65,34.29,OX40N,NTT,CERAMICS
48,JIOFIN,310.58,387.00,2.05,M-LC,89.25,218750.0,1344.0,52150.0,3.03,0.62,23.84,24.61,95.0,0.03,1.61,55.53,XY24,BTT,FINANCE
44,INDUSINDBK,1354.50,1800.00,81.30,L-MC,63.44,49080.0,-28126.0,53522.0,2.98,-36.43,109.05,32.89,253.0,-0.53,0.36,35.16,XR,NTT,BANKS
27,DIXON,14367.00,18940.15,-36.06,H-MC,21.51,117704.0,2768.0,33816.0,2.78,2.41,28.73,31.83,99.0,0.08,0.87,34.90,X40N,ATH,IT
2,ACC,2579.05,3906.00,-34.60,M-MC,35.48,140948.0,-47323.0,144190.0,2.61,-25.14,102.30,51.45,166.0,-0.33,1.04,7.16,XY24,BTT,CEMENT


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RAJESHEXPO,517.65,518.00,1835.61,M-SC,39.78,52804.0,-84373.0,84465.0,-1.99,-61.51,159.96,0.07,231.0,-1.00,0.39,31.33,OX40N,NTT,JEWELLERY
67,SAMMAANCAP,170.35,326.00,-173.86,M-SC,95.70,84510.0,-17700.0,111088.0,-1.76,-17.32,131.45,91.37,206.0,-0.16,0.62,37.19,XY25,NTT,FINANCE
17,BSOFT,628.85,844.81,-9.34,M-SC,61.29,76038.0,-34011.0,71803.0,-1.74,-30.91,94.43,34.34,200.0,-0.47,0.56,25.18,XR,ATH,IT
47,JCHAC,2282.24,2282.00,20008.70,M-SC,51.61,98246.0,-29559.0,29543.0,-1.55,-23.13,30.07,-0.01,222.0,-1.00,0.72,9.04,OX40N,NTT,AC
29,EASEMYTRIP,18.12,26.40,5.28,M-SC,6.45,101288.0,-75527.0,156328.0,-1.52,-42.72,154.34,45.70,190.0,-0.48,0.75,0.00,XY24,NTT,TRAVEL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UJJIVANSFB,52.77,53.0,52.86,M-SC,84.95,132138.0,-10341.0,10967.0,-0.33,-7.26,8.30,0.44,243.0,-0.94,0.97,55.96,OX40N,NTT,BANKS
79,TATAELXSI,7332.28,7332.0,-15.63,X-MC,33.33,88627.0,-14025.0,14021.0,0.89,-13.66,15.82,-0.00,58.0,-1.00,0.65,34.72,OX40N,NTT,IT
90,VIPIND,488.80,489.0,-212.52,H-SC,96.77,81140.0,-14176.0,14216.0,-0.14,-14.87,17.52,0.04,154.0,-1.00,0.60,60.04,OX40N,NTT,MISC
77,SYMPHONY,1306.42,1306.0,-33.55,M-SC,1.08,141650.0,-29491.0,29435.0,0.08,-17.23,20.78,-0.03,189.0,-1.00,1.04,1.81,OX40N,NTT,DURABLES
50,KANSAINER,299.63,340.0,-68.02,H-SC,23.12,223155.0,-46512.0,82835.0,1.58,-17.25,37.12,13.47,142.0,-0.56,1.65,12.87,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TCS,3794.03,4998.00,-17.74,X-LC,29.03,268328.0,-27606.0,121526.0,-0.05,-9.33,45.29,31.73,1.0,-0.23,1.98,7.40,X40,BTT,IT
45,INFY,1461.46,2275.00,-10.27,X-LC,56.99,287904.0,26303.0,119307.0,-0.47,10.05,41.44,55.67,2.0,0.22,2.12,16.77,X40,BTT,IT
37,HINDUNILVR,2413.81,2723.00,-17.84,X-LC,24.73,257544.0,-12803.0,47440.0,0.82,-4.74,18.42,12.81,6.0,-0.27,1.90,7.71,XY25,NTT,FMCG
59,NESTLEIND,2268.60,2755.00,-9.55,X-LC,32.26,286744.0,21318.0,35585.0,0.80,8.03,12.41,21.44,9.0,0.60,2.12,14.73,XY25,NTT,FMCG
16,BRITANNIA,4983.33,6446.05,16.86,X-LC,75.27,267766.0,38533.0,28758.0,-0.12,16.81,10.74,29.35,10.0,1.34,1.98,27.23,XY25,ATH,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,WHIRLPOOL,1167.49,2270.0,-36.92,M-SC,90.32,208078.0,36457.0,125617.0,0.56,21.24,60.37,94.43,198.0,0.29,1.53,53.57,XR,NTT,DURABLES
86,UNIONBANK,123.87,163.0,-2.93,M-LC,76.34,170357.0,29517.0,14974.0,1.15,20.96,8.79,31.59,89.0,1.97,1.26,47.98,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.40,M-LC,53.76,102659.0,24297.0,9732.0,1.16,31.01,9.48,43.43,77.0,2.50,0.76,37.97,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SAIL,130.64,228.00,89.46,M-MC,52.69,227821.0,2859.0,164806.0,-0.99,1.27,72.34,74.53,185.0,0.02,1.68,32.27,XY24,BTT,STEEL
19,CAMS,3643.00,5250.99,7.21,H-SC,81.72,120448.0,18444.0,26583.0,2.11,18.08,22.07,44.14,124.0,0.69,0.89,37.26,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,53.76,102659.0,24297.0,9732.0,1.16,31.01,9.48,43.43,77.0,2.50,0.76,37.97,XY24,NTT,MISC
48,JIOFIN,310.58,387.00,2.05,M-LC,89.25,218750.0,1344.0,52150.0,3.03,0.62,23.84,24.61,95.0,0.03,1.61,55.53,XY24,BTT,FINANCE
86,UNIONBANK,123.87,163.00,-2.93,M-LC,76.34,170357.0,29517.0,14974.0,1.15,20.96,8.79,31.59,89.0,1.97,1.26,47.98,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,WHIRLPOOL,1167.49,2270.00,-36.92,M-SC,90.32,208078.0,36457.0,125617.0,0.56,21.24,60.37,94.43,198.0,0.29,1.53,53.57,XR,NTT,DURABLES
93,WIPRO,243.46,420.00,-3.77,M-LC,78.49,164629.0,13684.0,95765.0,-1.11,9.07,58.17,72.51,69.0,0.14,1.21,13.40,XR,NTT,IT
42,INDIAMART,2327.09,4871.06,-46.96,H-SC,92.47,138250.0,14914.0,119918.0,-1.09,12.09,86.74,109.32,122.0,0.12,1.02,36.58,AR,ATH,MISC
58,NATIONALUM,189.63,247.44,-49.40,H-MC,67.74,101601.0,1097.0,29546.0,-0.98,1.09,29.08,30.49,97.0,0.04,0.75,33.85,MH,ATH,METALS
35,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,34.41,151624.0,711.0,59740.0,1.26,0.47,39.40,40.06,102.0,0.01,1.12,24.02,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,93.55,71578.0,-41971.0,82021.0,-1.03,-36.96,114.59,35.27,261.0,-0.51,0.53,75.19,XR,NTT,HOTELS
56,MASFIN,326.60,402.39,-17.80,H-SC,77.42,93330.0,-4650.0,27383.0,0.70,-4.75,29.34,23.21,137.0,-0.17,0.69,35.36,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,4.57,X-SC,98.92,97262.0,-3701.0,98235.0,-0.37,-3.67,101.00,93.63,64.0,-0.04,0.72,44.87,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,53.76,102659.0,24297.0,9732.0,1.16,31.01,9.48,43.43,77.0,2.50,0.76,37.97,XY24,NTT,MISC
5,APOLLOHOSP,6904.43,8285.00,-20.39,H-LC,50.54,108518.0,4952.0,15757.0,1.87,4.78,14.52,20.00,30.0,0.31,0.80,19.53,X40N,BTT,HEALTHCARE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7188.89,M-SC,100.00,79989.0,-13821.0,90436.0,-0.79,-14.73,113.06,81.67,242.0,-0.15,0.59,54.90,XR,NTT,CERAMICS
4,ALKYLAMINE,2347.98,4546.37,4.57,X-SC,98.92,97262.0,-3701.0,98235.0,-0.37,-3.67,101.00,93.63,64.0,-0.04,0.72,44.87,SR,ATH,CHEMICALS
49,JPPOWER,18.73,26.20,-31.87,L-SC,97.85,136617.0,-5225.0,61792.0,0.39,-3.68,45.23,39.88,259.0,-0.08,1.01,38.45,XY24,NTT,POWER
90,VIPIND,488.80,489.00,-212.52,H-SC,96.77,81140.0,-14176.0,14216.0,-0.14,-14.87,17.52,0.04,154.0,-1.00,0.60,60.04,OX40N,NTT,MISC
67,SAMMAANCAP,170.35,326.00,-173.86,M-SC,95.70,84510.0,-17700.0,111088.0,-1.76,-17.32,131.45,91.37,206.0,-0.16,0.62,37.19,XY25,NTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.05
1,25,42.05
2,50,70.41


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.39
LC    34.12
MC    21.45
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.95
X40      12.26
XY25     12.07
X40N     11.78
XR       10.23
OX40N     8.15
AR        7.82
MH        2.59
X5K       1.49
X200      1.41
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.55
M-SC    16.04
H-LC    12.55
X-LC    12.25
H-MC     8.57
M-LC     8.25
X-MC     7.57
M-MC     4.95
X-SC     4.34
L-SC     2.46
L-LC     1.07
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.22,-1.46,35.38
IT,11.81,-10.80,65.83
BANKS,7.90,-6.40,49.81
FINANCE,7.85,-13.24,54.22
MISC,6.60,-20.95,68.25
PAINTS,5.62,-14.60,37.18
ELECTRICAL,5.01,-3.37,48.14
HEALTHCARE,4.47,-6.39,37.46
AUTO,4.37,-14.10,64.28


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2734757.0,25
XR,1070457.0,13
AR,1011968.0,9
X40,655913.0,9
X40N,567461.0,11
XY25,482055.0,8
OX40N,322083.0,11
SR,180754.0,2
MH,148564.0,3


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2050422.0,21
M-SC,1764626.0,21
H-LC,567195.0,11
X-LC,515653.0,7
M-MC,496021.0,4
H-MC,473718.0,7
X-MC,424087.0,6
X-SC,329567.0,5
M-LC,325479.0,6


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,864854.0,6
M-SC,XY24,817658.0,7
H-SC,AR,520184.0,3
M-MC,XY24,384441.0,3
M-SC,XR,328584.0,4
X-LC,X40,240833.0,2
H-SC,X40N,235232.0,4
H-MC,XY24,225995.0,2
H-SC,XR,214841.0,3


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
